高斯分布
$$
    f(x) = \frac{1}{\sqrt{2\pi}\sigma}\exp\left ( -\frac{(x-\mu)^2}{2\sigma^2} \right )
$$

# 第6章 与学习相关的技巧

神经网络的学习中的一些重要观点：寻找最优权重参数的最优方法、权重参数的初始值、超参数的设定方法。
应对过拟合：权值衰减、Dropout等正则化方法。
Batch Normalization方法

## 6.1 参数的更新

神经网络的学习的目的是找到使损失函数的值尽可能小的参数。这是寻找最优参数的问题，解决这个问题的过程称为**最优化**（optimization）。

为了找到最优参数，我们将参数的梯度（导数）作为了线索。
使用参数的梯度，沿梯度方向更新参数，并重复这个步骤多次，从而逐渐靠
近最优参数，这个过程称为**随机梯度下降法**（stochastic gradient descent），简称SGD。

修饰变量的时候用 random。修饰过程的时候用 stochastic。

### 6.1.1 探险家的故事

朝着当前所在位置的坡度最大的方向前进，就是SGD的策略。

### 6.1.2 SGD

$$
    \textbf{W} \leftarrow  \textbf{W} - \eta \frac{\partial L}{\partial \textbf{W}}
    \tag{6.1}
$$

$\textbf{W}$ 权重参数
$\frac{\partial L}{\partial \textbf{W}}$ 损失函数关于 $\textbf{W}$ 的梯度。
$\eta$ 学习率，实际上会取0.01或0.001这些事先决定好的值。


$\leftarrow$ 表示用右边的值更新左边的值。

In [1]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr

    """
        params：权重参数 params['W1']
        grads：权重参数的梯度  grads['W1']
    """

    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

### 6.1.3 SGD的缺点

$$
    f(x,y) = \frac{1}{20}x^2 + y^2 \tag{6.2}
$$

![](../images/图6-1.PNG)

![](../images/图6-2.PNG)

![](../images/图6-3.PNG)

SGD的缺点是，如果函数的形状非均向（anisotropic），比如呈延伸状，搜索的路径就会非常低效。
SGD低效的根本原因是，梯度的方向并没有指向最小值的方向。

Momentum、AdaGrad、Adam这3种方法来取代SGD。

### 6.1.4 Momentum（动量）

Momentum是“动量”的意思，和物理有关。用数学式表示Momentum方法，如下所示。
$$
    \boldsymbol{v} \leftarrow \alpha \boldsymbol{v} - \eta \frac{\partial L}{\partial \textbf{W}} \tag{6.3}
    \\
    \textbf{W} \leftarrow \textbf{W} + \boldsymbol{v} \tag{6.4}
$$

Momentum的代码实现（源代码在[common/optimizer.py](/common/optimizer.py)中）

In [2]:
import numpy as np


class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None

    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)

        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

![](../images/图6-5.基于Momentum的最优化的更新路径.PNG)

图6-5.基于Momentum的最优化的更新路径

### 6.1.5 AdaGrad （Adaptive Gradient）

学习率过小，会导致学习花费过多的时间，
学习率过大，会导致学习发散而不能正确进行。

**学习率衰减**（learning rate decay）的方法，即随着学习的进行，使学习率逐渐减小。

$$
    \boldsymbol{h} \leftarrow \boldsymbol{h} + \frac{\partial L}{\partial \textbf{W}} \odot \frac{\partial L}{\partial \textbf{W}} \tag{6.5}
    \\
    \textbf{W} \leftarrow \textbf{W}  - \eta \frac{1}{\sqrt{\boldsymbol{h}}} \frac{\partial L}{\partial \textbf{W}}
    \tag{6.6}
$$

$\odot$ 表示对应矩阵元素的乘法。
$\boldsymbol{h}$ 保存了以前所有梯度值的平方和。
在更新参数时，通过乘以 $\frac{1}{\sqrt{h}}$，就可以调整学习的尺度。这意味着，
参数的元素中变动较大（被大幅更新）的元素的学习率将变小。也就是说，
可以按参数的元素进行学习率衰减，使变动大的参数的学习率逐渐减小。


In [3]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None

    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)

        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

![图6-6.基于AdaGrad的最优化的更新路径](../images/图6-6.基于AdaGrad的最优化的更新路径.PNG)
图6-6.基于AdaGrad的最优化的更新路径

### 6.1.6 Adam

Momentum参照小球在碗中滚动的物理规则进行移动，AdaGrad为参数的每个元素适当地调整更新步伐。
Adam融合了Momentum和AdaGrad的方法。

![图6-7.基于Adam的最优化的更新路径](../images/图6-7.基于Adam的最优化的更新路径.PNG)
图6-7.基于Adam的最优化的更新路径

### 6.1.7 使用那种更新方法

（目前）并不存在能在所有问题中都表现良好的方法。
[optimizer_compare_naive.py](../ch06/optimizer_compare_naive.py)

![](../images/图6-8.PNG)

![图6-8.最优化方法的比较：SGD、Momentum、AdaGrad、Adam](../images/图6-8.最优化方法的比较：SGD、Momentum、AdaGrad、Adam.PNG)
图6-8.最优化方法的比较：SGD、Momentum、AdaGrad、Adam

### 6.1.8 基于MNIST数据集的更新方法比较

![图6-9.基于MNIST数据集的4种更新方法的比较](../images/图6-9.基于MNIST数据集的4种更新方法的比较.PNG)
图6-9.基于MNIST数据集的4种更新方法的比较

以手写数字识别为例，比较SGD、Momentum、AdaGrad、Adam这4种方法，并确认不同的方法在学习进展上有多大程度的差异。
[optimizer_compare_mnist.py](../ch06/optimizer_compare_mnist.py)

![](../images/图6-9.png)

![](../images/图6-9.基于MNIST数据集的4种更新方法的比较.PNG)
图6-9 基于MNIST数据集的4种更新方法的比较：横轴表示学习的迭代次数（iteration），纵轴表示损失函数的值（loss）

## 6.2 权重的初始值

在神经网络的学习中，权重的初始值特别重要。实际上，设定什么样的
权重初始值，经常关系到神经网络的学习能否成功。

### 6.2.1 可以将权重初始值设置为0吗

抑制过拟合、提高泛化能力的技巧——权值衰减（weight decay）。
权值衰减就是一种以减小权重参数的值为目的进行学习的方法。
通过减小权重参数的值来抑制过拟合的发生。

将权重初始值设为0的话，将无法正确进行学习。

为了防止“权重均一化”（严格地讲，是为了瓦解权重的对称结构），必须随机生成初始值。

### 6.2.2 隐藏层的激活函数的分布

观察权重初始值是如何影响隐藏层的激活值的分布的。

向一个5层神经网络（激活函数使用sigmoid函数）传入随机生成的输入数据，用直方图绘制各层激活值的数据分布。

[weight_init_activation_histogram.py](../ch06/weight_init_activation_histogram.py)

![](../images/图6-10.使用标准差为1的高斯分布作为权重初始值时的各层激活值的分布1.png)
图6-10　使用标准差为1的高斯分布作为权重初始值时的各层激活值的分布

从图6-10可知，各层的激活值呈偏向0和1的分布。这里使用的sigmoid
函数是S型函数，随着输出不断地靠近0（或者靠近1），它的导数的值逐渐接
近0。因此，偏向0和1的数据分布会造成反向传播中梯度的值不断变小，最
后消失。这个问题称为**梯度消失**（gradient vanishing）。层次加深的深度学习
中，梯度消失的问题可能会更加严重。



    # w = np.random.randn(node_num, node_num) * 1
    w = np.random.randn(node_num, node_num) * 0.01

![](../images/图6-11.使用标准差为0.01的高斯分布作为权重初始值时的各层激活值的分布1.png)
图6-11　使用标准差为0.01的高斯分布作为权重初始值时的各层激活值的分布

这次呈集中在0.5附近的分布。因为不像刚才的例子那样偏向0和1，所
以不会发生梯度消失的问题。但是，激活值的分布有所偏向，说明在表现力
上会有很大问题。为什么这么说呢？因为如果有多个神经元都输出几乎相同
的值，那它们就没有存在的意义了。比如，如果100个神经元都输出几乎相
同的值，那么也可以由1个神经元来表达基本相同的事情。因此，激活值在
分布上有所偏向会出现“表现力受限”的问题。

Xavier的论文中，为了使各层的激活值呈现出具有相同广度的分布，推
导了合适的权重尺度。推导出的结论是，如果前一层的节点数为n，则初始
值使用标准差为 $\frac{1}{\sqrt{n}}$ 的分布

![](../images/图6-12.Xavier初始值.PNG)
图6-12 Xavier初始值：与前一层有n个节点连接时，初始值使用标准差为$\frac{1}{\sqrt{n}}$ 的分布

    w = np.random.randn(node_num, node_num) / np.sqrt(node_num)

使用Xavier初始值后的结果如图6-13所示。从这个结果可知，越是后
面的层，图像变得越歪斜，但是呈现了比之前更有广度的分布。因为各层间
传递的数据有适当的广度，所以sigmoid函数的表现力不受限制，有望进行
高效的学习。

![](../images/图6-13.使用Xavier初始值作为权重初始值时的各层激活值的分布.PNG)
图6-13　使用Xavier初始值作为权重初始值时的各层激活值的分布


### 6.2.3 ReLU的权重初始值

Xavier初始值是以激活函数是线性函数为前提而推导出来的。

![图6-14.激活函数使用ReLU时,不同权重初始值的激活值分布的变化](../images/图6-14.激活函数使用ReLU时,不同权重初始值的激活值分布的变化.PNG)
图6-14.激活函数使用ReLU时,不同权重初始值的激活值分布的变化

当激活函数使用ReLU时，权重初始值使用He初始值，
当激活函数为sigmoid或tanh等S型曲线函数时，初始值使用Xavier初始值。
这是目前的最佳实践。

### 6.2.4 基于MNIST数据集的权重初始值的比较

基于std = 0.01、Xavier初始值、He初始值进行实验
[weight_init_compare.py](../ch06/weight_init_compare.py)

![](../images/图6-15.png)

![](../images/图6-15.基于MNIST数据集的权重初始值的比较：横轴是学习的迭代次数（iterations），纵轴是损失函数的值（loss）.PNG)
图6-15 基于MNIST数据集的权重初始值的比较：横轴是学习的迭代次数（iterations），纵轴是损失函数的值（loss）

在神经网络的学习中，权重初始值非常重要。很多时候权重初始值的设定关系到神经网络的学习能否成功。
权重初始值的重要性容易被忽视，而任何事情的开始（初始值）总是关键的。

## 6.3 Batch Normalization

如果设定了合适的权重初始值，则各层的激活值分布会有适当的广度，从而可以顺利地进行学习。

Batch Normalization 方法：为了使各层拥有适当的广度，“强制性”地调整激活值的分布。

### 6.3.1 Bath Normalization 的算法

Batch Norm的优点
* 可以使学习快速进行（可以增大学习率）
* 不那么依赖初始值（对于初始值不用那么神经质）
* 抑制过拟合（降低Dropout等的必要性）

Batch Norm的思路是调整各层的激活值分布使其拥有适当的广度。
要向神经网络中插入对数据分布进行正规化的层，即Batch Normalization层。

![](../images/图6-16.使用了Batch%20Normalization的神经网络的例子.PNG)
图6-16.使用了Batch Normalization的神经网络的例子

Batch Norm 以进行学习时的mini-batch为单位，按mini-bath进行正规化。
进行使数据集分布的均值为0、方差为1的正规化。

用数学式表示：
$$
    \mu_B \leftarrow \frac{1}{m}\sum_{i=1}^{m}x_i
    \\
    \sigma_B^2 \leftarrow \frac{1}{m}\sum_{i=1}^{m}(x_i - \mu_B)^2
    \\
    \hat{x_i} \leftarrow \frac{x_i - \mu_B}{\sqrt{\sigma_B^2} + \varepsilon }
    \\
    \tag{6.7}
$$

这里对mini-batch的 $m$ 个输入数据的集合 $B = \left \{ x_1, x_2, ... , x_m \right \}$ 求均值 $\mu_B$ 和方差 $\sigma_B^2$ 。
然后，对输入数据进行均值为0、方差为1（合适的分布）的正规化。
式（6.7）中的 $\varepsilon$ 是一个微小值（比如，10e-7等），它是为了防止出现除以0的情况


接着，Batch Norm层会对正规化后的数据进行缩放和平移的变换，用
数学式可以如下表示

$$
    y_i \leftarrow \gamma\hat{x_i} + \beta \tag{6.8}
$$

![](../images/图6-17.Batch%20Normalization的计算图.PNG)
图6-17 Batch Normalization的计算图

Frederik Kratzert 的博客“Understanding the backward pass through Batch Normalization Layer”
https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html

[batch_norm_test.py](../ch06/batch_norm_test.py)

![](../images/图6-18.基于Batch%20Norm的效果.PNG)
图6-18.基于Batch Norm的效果：使用Batch Norm后，学习进行得更快了

![](../images/图6-19.PNG)
图6-19.图中的实线是使用了Batch Norm时的结果，虚线是没有使用Batch Norm时的结果：图的标题处标明了权重初始值的标准差

## 6.4 正则化

**过拟合**指的是只能拟合训练数据，但不能很好地拟合不包含在训练数据中的其他数据的状态。

### 6.4.1 过拟合

发生过拟合的原因：
* 模型拥有大量参数、表现力强
* 训练数据少

[overfit_weight_decay.py](../ch06/overfit_weight_decay.py)

![](../images/图6-20.训练数据（train）和测试数据（test）的识别精度的变化.PNG)
图6-20　训练数据（train）和测试数据（test）的识别精度的变化

### 6.4.2 权值衰减

**权值衰减**是一种抑制过拟合的方法。通过在学习的过程中对大的权重进行惩罚，来抑制拟合。
很多过拟合原本就是因为权重取值过大才发生的。

$L2$ 范数的权值衰减 $\frac{1}{2}\lambda\textbf{W}^2$

对于所有权重，权值衰减方法都会为损失函数加上 。因此，在求权重梯度的计算中，要为之前的误差反向传播法的结果加上正则化项的导数 $\lambda\textbf{W}$。

![](../images/图6-21.使用了权值衰减的训练数据（train）和测试数据（test）的识别精度的变化.PNG)
图6-21　使用了权值衰减的训练数据（train）和测试数据（test）的识别精度的变化

过拟合受到了抑制

$L1$ 范数：各个元素绝对值之和。
$\left| w_1 \right| + \left| w_1 \right| + \cdots + \left| w_n \right|$
$L2$ 范数：各个元素的平方和。
$ \sqrt{w_1^2 + w_2^2 + \cdots + w_n^2} $
$L\infty$ 范数：也称为Max范数，各个元素中最大的那一个。

### 6.4.3 Dropout

Dropout是一种在学习过程中随机删除神经元的方法。

![](../images/图6-23.左边没有使用Dropout，右边使用了Dropout（dropout_rate=0.15）.PNG)
图6-23　左边没有使用Dropout，右边使用了Dropout（dropout_rate=0.15）


In [ ]:
class Dropout:
    def __init__(self, dropout_ratio):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask = np.random.random(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
        return dout * self.mask

集成学习：是让多个模型单独进行学习，推理时再取多个模型的输出的平均值。
Dropout将集成学习的效果（模拟地）通过一个网络实现了。

## 6.5 超参数的验证

**超参数**（hyper-parameter）：各层的神经元数量、batch大小、参数更新时的学习率或权值衰减等。

### 6.5.1 验证数据

不能使用测试数据评估超参数的性能。
因为如果使用测试数据调整超参数，超参数的值会对测试数据发生过拟合。

调整超参数时，必须使用超参数专用的确认数据。
拥有调整超参数的数据，一般称为**验证数据**（validation data）。

数据集分类：训练数据、验证数据、测试数据

训练数据用于参数（权重和偏置）的学习，
验证数据用于超参数的性能评估。
为了确认泛化能力，要在最后使用（比较理想的是只用一次）测试数据。

如果是MNIST数据集，获得验证数据的最简单的方法就是从训练数据中事先分割20%作为验证数据：

In [ ]:
from common.util import shuffle_dataset
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist()
# 打乱训练数据
x_train, t_train = shuffle_dataset(x_train, t_train)
# 分割验证数据
validation_rate = 0.20
validation_num = int(x_train.shape[0] * validation_rate)

x_val = x_train[:validation_num]
t_val = t_train[:validation_num]
x_train = x_train[validation_num:]
t_train = t_train[validation_num:]

### 6.5.2 超参数的最优化

超参数的最优化：

* 步骤0
设定超参数的范围。

* 步骤1
从设定的超参数范围中随机采样。

* 步骤2
使用步骤1中采样到的超参数的值进行学习，通过验证数据评估识别精度（但是要将epoch设置得很小）。

* 步骤3
重复步骤1和步骤2（100次等），根据它们的识别精度的结果，缩小超参数的范围

这个方法与其说是科学方法，倒不如说有些实践者的经验的感觉。在超
参数的最优化中，如果需要更精炼的方法，可以使用**贝叶斯最优化**（Bayesian optimization）。

贝叶斯最优化运用以贝叶斯定理为中心的数学理论，能够更加严密、高效地进行最优化。
贝叶斯最优化（Bayesian optimization）：论文 [Practical Bayesian Optimization of Machine Learning Algorithms](https://arxiv.org/abs/1206.2944)

### 6.5.3　超参数最优化的实现

[hyperparameter_optimization.py](../ch06/hyperparameter_optimization.py)


![图6-24.实线是验证数据的识别精度，虚线是训练数据的识别精度](../images/图6-24.实线是验证数据的识别精度，虚线是训练数据的识别精度.PNG)
图6-24.实线是验证数据的识别精度，虚线是训练数据的识别精度

## 6.6 小结

* 参数的更新方法，除了SGD之外，还有Momentum、AdaGrad、Adam等方法。
* 权重初始值的赋值方法对进行正确的学习非常重要。
* 作为权重初始值，Xavier初始值、He初始值等比较有效。
* 通过使用Batch Normalization，可以加速学习，并且对初始值变得健壮。
* 抑制过拟合的正则化技术有权值衰减、Dropout等。
* 逐渐缩小“好值”存在的范围是搜索超参数的一个有效方法。